https://towardsdatascience.com/temporal-fusion-transformer-time-series-forecasting-with-deep-learning-complete-tutorial-d32c1e51cd91

In [130]:
import pandas as pd

In [131]:
df = pd.read_csv('../../data/df_no24.csv')
df.head()

,Unnamed: 0,date,sp500,cpi,nasdaq,gdp,unrate,usdeur,usdgbp
0,0,2014-02-18,1840.760010,235.547,3675.0,17197.738,6.7,0.72993,0.59661
1,1,2014-02-19,1828.750000,235.547,3654.5,17197.738,6.7,0.72969,0.59809
2,2,2014-02-20,1839.780029,235.547,3671.5,17197.738,6.7,0.72664,0.59930
3,3,2014-02-21,1836.250000,235.547,3664.5,17197.738,6.7,0.72831,0.59967
4,4,2014-02-24,1847.609985,235.547,3688.0,17197.738,6.7,0.72880,0.60018


In [132]:
data = df[['date', 'sp500', 'cpi', 'gdp', 'unrate', 'usdeur', 'usdgbp']] 
data.head()

,date,sp500,cpi,gdp,unrate,usdeur,usdgbp
0,2014-02-18,1840.760010,235.547,17197.738,6.7,0.72993,0.59661
1,2014-02-19,1828.750000,235.547,17197.738,6.7,0.72969,0.59809
2,2014-02-20,1839.780029,235.547,17197.738,6.7,0.72664,0.59930
3,2014-02-21,1836.250000,235.547,17197.738,6.7,0.72831,0.59967
4,2014-02-24,1847.609985,235.547,17197.738,6.7,0.72880,0.60018


In [133]:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.asfreq('D')
#data = data.sort_index()
data.head(20)

/var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/ipykernel_9126/3566864405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')


,sp500,cpi,gdp,unrate,usdeur,usdgbp
date,,,,,,
2014-02-18,1840.760010,235.547,17197.738,6.7,0.72993,0.59661
2014-02-19,1828.750000,235.547,17197.738,6.7,0.72969,0.59809
2014-02-20,1839.780029,235.547,17197.738,6.7,0.72664,0.59930
2014-02-21,1836.250000,235.547,17197.738,6.7,0.72831,0.59967
2014-02-22,NaN,NaN,NaN,NaN,NaN,NaN
2014-02-23,NaN,NaN,NaN,NaN,NaN,NaN
2014-02-24,1847.609985,235.547,17197.738,6.7,0.72880,0.60018
2014-02-25,1845.119995,235.547,17197.738,6.7,0.72791,0.60120
2014-02-26,1845.160034,235.547,17197.738,6.7,0.72801,0.60035


In [134]:
data.index = data.index.strftime('%d-%m-%Y %H:%M:%S')
data.head()

,sp500,cpi,gdp,unrate,usdeur,usdgbp
date,,,,,,
18-02-2014 00:00:00,1840.760010,235.547,17197.738,6.7,0.72993,0.59661
19-02-2014 00:00:00,1828.750000,235.547,17197.738,6.7,0.72969,0.59809
20-02-2014 00:00:00,1839.780029,235.547,17197.738,6.7,0.72664,0.59930
21-02-2014 00:00:00,1836.250000,235.547,17197.738,6.7,0.72831,0.59967
22-02-2014 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
data.fillna(method='ffill', inplace=True, limit=None, axis=None, downcast=None)
data.head()

/var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/ipykernel_9126/2589696618.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True, limit=None, axis=None, downcast=None)
/var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/ipykernel_9126/2589696618.py:1: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  data.fillna(method='ffill', inplace=True, limit=None, axis=None, downcast=None)


,sp500,cpi,gdp,unrate,usdeur,usdgbp
date,,,,,,
18-02-2014 00:00:00,1840.760010,235.547,17197.738,6.7,0.72993,0.59661
19-02-2014 00:00:00,1828.750000,235.547,17197.738,6.7,0.72969,0.59809
20-02-2014 00:00:00,1839.780029,235.547,17197.738,6.7,0.72664,0.59930
21-02-2014 00:00:00,1836.250000,235.547,17197.738,6.7,0.72831,0.59967
22-02-2014 00:00:00,1836.250000,235.547,17197.738,6.7,0.72831,0.59967


In [137]:
df_list = []
df = data

for label in df:
    ts = df[label]
    tmp = pd.DataFrame({'value': ts})
    tmp['date'] = ts.index
    tmp['ts_id'] = label
    df_list.append(tmp)

In [138]:
df_list[0].head()

,value,date,ts_id
date,,,
18-02-2014 00:00:00,1840.760010,18-02-2014 00:00:00,sp500
19-02-2014 00:00:00,1828.750000,19-02-2014 00:00:00,sp500
20-02-2014 00:00:00,1839.780029,20-02-2014 00:00:00,sp500
21-02-2014 00:00:00,1836.250000,21-02-2014 00:00:00,sp500
22-02-2014 00:00:00,1836.250000,22-02-2014 00:00:00,sp500


In [141]:
df = pd.concat(df_list).reset_index(drop=True)
df.tail()

,value,date,ts_id
21607,0.78613,25-12-2023 00:00:00,usdgbp
21608,0.79410,26-12-2023 00:00:00,usdgbp
21609,0.79430,27-12-2023 00:00:00,usdgbp
21610,0.79295,28-12-2023 00:00:00,usdgbp
21611,0.79134,29-12-2023 00:00:00,usdgbp


In [ ]:
from pytorch_forecasting import TimeSeriesDataSet, GroupNormalizer

#Hyperparameters
#batch size=64
#number heads=4, hidden sizes=160, lr=0.001, gr_clip=0.1

max_prediction_length = 24
max_encoder_length = 7*24
training_cutoff = df["date"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.hours_from_start <= training_cutoff],
    time_idx="date",
    target="value",
    group_ids=["ts_id"],
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["ts_id"],
    time_varying_known_reals=[],
    time_varying_unknown_reals=['power_usage'],
    target_normalizer=GroupNormalizer(
        groups=["consumer_id"], transformation="softplus"
    ),  # we normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)


validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

# create dataloaders for  our model
batch_size = 64 
# if you have a strong GPU, feel free to increase the number of workers  
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)